# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [ ]:
# TODO: Create a .env file with the following variables
# OPENAI_API_KEY="YOUR_KEY"
# CHROMA_OPENAI_API_KEY="YOUR_KEY"
# TAVILY_API_KEY="YOUR_KEY"

# This cell can't create secrets for you; it just helps validate setup.
if os.path.exists(".env"):
    print(".env found")
else:
    print(".env not found (create it in project/starter before running the next cell)")

In [ ]:
# TODO: Load environment variables
# load_dotenv()

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY") or OPENAI_API_KEY
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

assert OPENAI_API_KEY is not None, "Missing OPENAI_API_KEY in .env"
assert CHROMA_OPENAI_API_KEY is not None, "Missing CHROMA_OPENAI_API_KEY (or OPENAI_API_KEY) in .env"
assert TAVILY_API_KEY is not None, "Missing TAVILY_API_KEY in .env"

### VectorDB Instance

In [ ]:
# TODO: Instantiate your ChromaDB Client
# Choose any path you want
# chroma_client = chromadb.PersistentClient(path="chromadb")

from pathlib import Path

CHROMA_DIR = Path(os.getenv("CHROMA_PATH") or "chromadb").resolve()
CHROMA_DIR.mkdir(parents=True, exist_ok=True)

chroma_client = chromadb.PersistentClient(path=str(CHROMA_DIR))
print(f"ChromaDB path: {CHROMA_DIR}")

### Collection

In [ ]:
# TODO: Pick one embedding function
# If picking something different than openai, 
# make sure you use the same when loading it
# embedding_fn = embedding_functions.OpenAIEmbeddingFunction()

try:
    embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=CHROMA_OPENAI_API_KEY,
        model_name="text-embedding-3-small",
    )
except TypeError:
    embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=CHROMA_OPENAI_API_KEY)

embedding_fn

In [ ]:
# TODO: Create a collection
# Choose any name you want
# collection = chroma_client.create_collection(
#    name="udaplay",
#    embedding_function=embedding_fn
#)

COLLECTION_NAME = "udaplay"
try:
    chroma_client.delete_collection(name=COLLECTION_NAME)
except Exception:
    pass

collection = chroma_client.create_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_fn,
 )

collection

### Add documents

In [ ]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )